In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key="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",
    model="llama3-8b-8192",
  
)


In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/cdn-security-engineer-waf/job/R-48004")

page_data=loader.load().pop().page_content

print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.























CDN Security Engineer (WAF)










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu



In [4]:
loader_multiple_pages = WebBaseLoader(
    ["https://www.goldmansachs.com/careers/students/programs-and-internships/india/summer-analyst-program",
      "https://careers.nike.com/cdn-security-engineer-waf/job/R-48004"]
)

docs = loader_multiple_pages.load()




In [5]:
from langchain.prompts import PromptTemplate
from bs4 import BeautifulSoup

html_content_1 = docs[0].page_content
html_content_2 = docs[1].page_content

# Parse the HTML content using BeautifulSoup
soup_1 = BeautifulSoup(html_content_1, "html.parser")
soup_2 = BeautifulSoup(html_content_2, "html.parser")

# Extract text from the HTML
text_1 = soup_1.get_text()
text_2 = soup_2.get_text()

# Create a PromptTemplate for extracting information
promt_extract = PromptTemplate.from_template(
    """
    scrapped text FROM website:
    {job_posting_}

    Return ONLY the JSON format with no additional explanation or text 
Extract the following details from the scrapped text FROM websites below:

    1. **Role**: The job title or position.
    2. **Experience**: The years of experience required or preferred.
    3. **Remote/Hybrid**: Whether the job is remote or hybrid, or if it's specified in the posting.
    4. **Skills**: Key skills required for the role, including technical and soft skills.
    5. **Description**: A short description of the role and responsibilities.

    Return ONLY the JSON format with no additional explanation or text
    
    {{
        "role": "<Job Title>",
        "experience": "<Experience Required>",
        "remote_or_hybrid": "<Remote/Hybrid>",
        "skills": "<Skills>",
        "description": "<Role Description>"
    }}
    
    
    
   
    VALID JSON NO PREAMBLE
    """
)

chain_extract = promt_extract | llm

res_1 = chain_extract.invoke(input={'job_posting_': text_1})
res_2 = chain_extract.invoke(input={'job_posting_': text_2})
output1 = res_1.content
output2= res_2.content


print(output1)
print(output2)




{
    "role": "Summer Analyst",
    "experience": "Students graduating from a Bachelors or Masters degree program December 2025 - May 2026",
    "remote_or_hybrid": "Not specified",
    "skills": "Not specified",
    "description": "An eight to ten week summer internship for undergraduate and postgraduate students. You will be fully immersed in our day-to-day activities."
}
{
    "role": "CDN Security Engineer (WAF)",
    "experience": "At least five (5) or more years of relevant experience",
    "remote_or_hybrid": "Flexible 4/1 Hybrid schedule (Mon-Thurs in the office, Fridays optional at home)",
    "skills": "CDN concepts, Akamai's suite of tools and security offerings, API security requirements, web attack vectors, threat indicators, cloud infrastructure, security, or a related field",
    "description": "Design, implement, and sustain Nike's cloud infrastructure and security posture, with a focus on Content Delivery Network (CDN) security"
}


In [6]:
from langchain_core.output_parsers import JsonOutputParser

json_parsser= JsonOutputParser()
jrp1=json_parsser.parse(output1)
jrp2=json_parsser.parse(output2)


In [7]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [8]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [9]:
job=jrp2
job

{'role': 'CDN Security Engineer (WAF)',
 'experience': 'At least five (5) or more years of relevant experience',
 'remote_or_hybrid': 'Flexible 4/1 Hybrid schedule (Mon-Thurs in the office, Fridays optional at home)',
 'skills': "CDN concepts, Akamai's suite of tools and security offerings, API security requirements, web attack vectors, threat indicators, cloud infrastructure, security, or a related field",
 'description': "Design, implement, and sustain Nike's cloud infrastructure and security posture, with a focus on Content Delivery Network (CDN) security"}

In [10]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/kotlin-android-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}]]

In [11]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        Generate a professional, concise cold email from Atharv Bhujannavar, a B.Tech graduate, who is reaching out to potential clients or companies. The goal is to introduce himself, express interest in collaborating or offering value through his skills, and initiate a conversation.

Include:

A warm, personalized greeting

A short introduction of Atharv (as a recent B.Tech graduate)

A clear value proposition or reason for reaching out

A call to action (e.g., a meeting, call, or reply)

Keep the tone polite, confident, and professional

Output a subject line and the full email body.Also add the most relevant ones from the following links to showcase my portfolio: {link_list}
            Remember you are atharv bhujannavar.
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Here is a professional, concise cold email:

**Subject: Exploring Opportunities in CDN Security**

Dear Hiring Manager,

I hope this email finds you well. My name is Atharv Bhujannavar, a recent B.Tech graduate with a strong passion for cloud infrastructure and security. I came across Nike's job posting for a CDN Security Engineer (WAF) and was impressed by the company's commitment to innovation and security.

As a recent graduate, I'm eager to apply my skills and knowledge to real-world projects and collaborate with experienced professionals like yourself. With a solid foundation in CDN concepts, Akamai's suite of tools, and API security requirements, I believe I can bring value to your team.

I'd love to discuss how my skills align with Nike's needs and explore potential opportunities for collaboration. Would you be available for a brief call to discuss further?

Please find some examples of my work in the following links: [https://example.com/kotlin-android-portfolio](https://exampl